In [1]:
import json
import gzip

from utils import download_url

#raw_url= "https://ml.materialsproject.org/projects/matbench_mp_is_metal.json.gz"
#raw_dir= "../data/matbench/mp_is_metal"
#download_url(raw_url, raw_dir)


json_filename = "../data/matbench/mp_is_metal/matbench_mp_is_metal.json.gz"


with gzip.open(json_filename, 'r') as fin:        # 4. gzip
    json_bytes = fin.read()                      # 3. bytes (i.e. UTF-8)

json_str = json_bytes.decode('utf-8')            # 2. string (i.e. JSON)
data = json.loads(json_str) 


from pymatgen.core.structure import Structure


struct= Structure.from_dict(data["data"][0][0])

In [18]:
struct.atomic_numbers, struct.get_space_group_info(), struct.charge, struct.density, struct.distance_matrix

struct.as_dataframe()

,Species,a,b,c,x,y,z,magmom
0,(K),0.000888,0.002582,0.005358,-0.008126,0.024760,-0.016981,-0.005
1,(K),0.504645,0.002727,0.005671,-0.771076,-2.736649,-1.500623,-0.005
2,(K),0.496497,0.498521,0.493328,-2.859789,0.002464,-2.016825,-0.005
3,(K),0.496657,0.994989,0.493552,-5.684794,-0.001713,-0.557008,-0.005
4,(Mn),0.993288,0.493319,0.986797,-2.853450,0.008701,-5.517640,4.039
5,(Mn),0.005896,0.005967,0.512019,0.713301,2.801394,-2.099078,4.039
6,(Mn),0.005704,0.505997,0.511635,-2.132331,2.795754,-0.625252,4.039
7,(Mn),0.493721,0.493503,0.987141,-2.097738,2.750766,-4.048060,4.039
8,(O),0.190852,0.749673,0.499250,-3.817505,1.709431,-0.402260,-0.097
9,(O),0.749870,0.691083,0.999719,-3.591226,1.413578,-4.272077,-0.097


## Matminer

In [13]:


%matplotlib inline
from matplotlib import pyplot as plt
from matminer.datasets import load_dataset
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers.composition import ElementProperty, Stoichiometry, ValenceOrbital, IonProperty
from matminer.featurizers.structure import (SiteStatsFingerprint, StructuralHeterogeneity,
                                            ChemicalOrdering, StructureComposition, MaximumPackingEfficiency)
from matminer.featurizers.conversions import DictToObject
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from scipy import stats
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd

In [3]:


featurizer = MultipleFeaturizer([
    SiteStatsFingerprint.from_preset("CoordinationNumber_ward-prb-2017"),
    StructuralHeterogeneity(),
    ChemicalOrdering(),
    MaximumPackingEfficiency(),
    SiteStatsFingerprint.from_preset("LocalPropertyDifference_ward-prb-2017"),
    StructureComposition(Stoichiometry()),
    StructureComposition(ElementProperty.from_preset("magpie")),
    StructureComposition(ValenceOrbital(props=['frac'])),
    StructureComposition(IonProperty(fast=True))
])

In [24]:
X[0,:]

array([6.27134027e+00, 9.02101351e+00, 2.74967324e+00, 7.37003555e+00,
       8.24896244e-01, 6.08243743e-02, 1.12169330e+00, 9.03899528e-01,
       8.92173175e-02, 1.35354787e-01, 4.61374697e-02, 1.12876867e-01,
       1.34657734e-02, 1.22116296e-01, 7.83641060e-01, 5.43588854e-01,
       3.93978380e-01, 3.88073327e-01, 8.85542642e+00, 1.66635663e+01,
       7.80813987e+00, 1.27072763e+01, 1.99832453e+00, 3.54000982e+01,
       6.77150510e+01, 3.23149528e+01, 5.25897272e+01, 8.59411696e+00,
       1.86753168e+01, 3.82305607e+01, 1.95552440e+01, 2.82486915e+01,
       4.99774922e+00, 2.52141982e+02, 1.45507306e+03, 1.20293108e+03,
       8.22237900e+02, 3.16398075e+02, 8.90544218e+00, 1.20240124e+01,
       3.11857019e+00, 1.06438552e+01, 8.69078145e-01, 1.58320941e+00,
       1.93937130e+00, 3.56161891e-01, 1.78364106e+00, 1.00149360e-01,
       7.26930476e+01, 1.10046674e+02, 3.73536259e+01, 9.30225835e+01,
       1.01642668e+01, 1.85424229e+00, 2.09233539e+00, 2.38093106e-01,
      

In [14]:
## creating small dataframe

structures = []
target=[]

for i in range(10):
    structures.append(Structure.from_dict(data["data"][0][0]))
    target.append(data["data"][0][1])
    
df= pd.DataFrame({"structure":structures, "target":target})

In [17]:


dto = DictToObject(target_col_id='structure', overwrite_data=True)
data = dto.featurize_dataframe(df, 'structure')


DictToObject:   0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
## featurizing one structure

%%time
print('Total number of features:', len(featurizer.featurize(data['structure'][0])))
print('Number of sites in structure:', len(data['structure'][0]))


Total number of features: 273
Number of sites in structure: 16
CPU times: user 1.09 s, sys: 7.25 ms, total: 1.1 s
Wall time: 1.12 s


In [19]:


%%time
X = featurizer.featurize_many(data['structure'], ignore_errors=True)


MultipleFeaturizer:   0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 39.7 ms, sys: 377 ms, total: 417 ms
Wall time: 18 s


In [22]:
X = np.array(X)

X.shape

(10, 273)

In [23]:
X

array([[6.27134027, 9.02101351, 2.74967324, ..., 0.        , 0.82023413,
        0.18414808],
       [6.27134027, 9.02101351, 2.74967324, ..., 0.        , 0.82023413,
        0.18414808],
       [6.27134027, 9.02101351, 2.74967324, ..., 0.        , 0.82023413,
        0.18414808],
       ...,
       [6.27134027, 9.02101351, 2.74967324, ..., 0.        , 0.82023413,
        0.18414808],
       [6.27134027, 9.02101351, 2.74967324, ..., 0.        , 0.82023413,
        0.18414808],
       [6.27134027, 9.02101351, 2.74967324, ..., 0.        , 0.82023413,
        0.18414808]])